### This script performs epoching of the raw eeg data based on the on/off beat timing obtained by syllable_time.ipynb
The on/off_beats2 are obtained as the difference between two neighbour syllable start time.

In [1]:
import os

import numpy as np
from scipy.io import wavfile
from dSPEECH.config import *
import json

2024-08-10 20:26:35 - Start Program

pre_all: Running from CMD.
common_dl.py: Using CUDA.


In [2]:
sid=2
folder=meta_dir+'SEEG'+str(sid)+'/forced_alignment/' #alignment/'
filename=folder+'/syllable_time.npy'
result=np.load(filename, allow_pickle=True).item()

In [3]:
on_beats=result['on_beats']
off_beats=result['off_beats']
on_beats2=result['on_beats2']
off_beats2=result['off_beats2']
on_beats_phones=result['on_beats_phones']
off_beats_phones=result['off_beats_phones']

In [5]:
dur_on=[]
for sen in on_beats2:
    dur_sen=[i[1]-i[0] for i in sen]
    dur_on.append(dur_sen)

dur_off=[]
for sen in off_beats2:
    dur_sen=[i[1]-i[0] for i in sen]
    dur_off.append(dur_sen)

In [8]:
dur_off[36]

[0.15999999999999992,
 0.2599999999999998,
 0.2999999999999998,
 0.29000000000000004]

In [9]:
a=[i for i,dur_sen in enumerate(dur_on) if all(dur_sen)==0] # 0 duration
b=[i for i,dur_sen in enumerate(dur_off) if all(dur_sen)==0]
print('Onbeats syllables with 0 durations: '+','.join(str(i) for i in a)+'; Offbeats syllables with 0: '+','.join(str(i) for i in b))

Onbeats syllables with 0 durations: ; Offbeats syllables with 0: 


In [7]:
shortest_on=min([min(dur_sen) for dur_sen in dur_on if all(dur_sen) != 0])
shortest_off=min([min(dur_sen) for dur_sen in dur_off if all(dur_sen) != 0])
print('Shortest onbeat syllable: '+str(shortest_on)+'; shortest of offbeat: '+str(shortest_off)+'.')

Shortest onbeat syllable: 0.009999999999999787; shortest of offbeat: 0.009999999999999787.


In [8]:
shortest_on=[i for i, dur_sen in enumerate(dur_on) if shortest_on in dur_sen] # shortest: 37
shortest_off=[i for i, dur_sen in enumerate(dur_off) if shortest_off in dur_sen] # shortest: [32, 62, 63, 92]

In [9]:
shortest_on

[37]

In [10]:
shortest_off

[32, 62, 92]

In [13]:
modality='SEEG'
sid=2 # 1/2
sf=1024
## load epochs and sentences
filename=data_dir+'processed/'+modality+str(sid)+'/'+modality+str(sid)+'-epo.fif'
epochs=mne.read_epochs(filename)
filename2=data_dir+'processed/'+modality+str(sid)+'/sentences.npy'
sentences=np.load(filename2,allow_pickle=True)

# two events: 1 stands for 'TRIG[001]:1' and 2 stands for 'TRIG[001]:1:inserted'
epoch1=epochs['1'].get_data() # (99, 150, 15361)
epoch2=epochs['2'].get_data() # (4, 150, 15361)
epochs=np.concatenate((epoch1,epoch2),axis=0) # (100 trials, 150 channels, 15361 time)


In [14]:
epochs=epochs[:,:,int(5*sf):int(10*sf)]

In [15]:
ons_tmp=[]
offs_tmp=[]
for i in range(100):
    on_sent=[] # per sentence
    off_sent=[]
    for on_syllable,off_syllable in zip(on_beats2[i],off_beats2[i]):
        start=int(sf*on_syllable[0])
        end=int(sf*on_syllable[1])
        on_sent.append(epochs[i,:,start:end])
        
        start2=int(sf*off_syllable[0])
        end2=int(sf*off_syllable[1])
        off_sent.append(epochs[i,:,start2:end2])
    ons_tmp.append(on_sent)
    offs_tmp.append(off_sent)
    

In [43]:
ons=[ele.transpose() for eles in ons_tmp for ele in eles] # transpose(): when the leading dimmen is different, no problem. but if the tailing dimmen is different, then np.save failed.
offs=[ele.transpose() for eles in offs_tmp for ele in eles]
print('Len of ons: '+str(len(ons))+'; len of offs: '+str(len(offs))+'.')

Len of ons: 366; len of offs: 366.


In [44]:
result={}
result['ons']=np.array(ons,dtype=object)
result['offs']=np.array(offs,dtype=object)
filename=data_dir+'processed/'+modality+str(sid)+'/on_off_beat/on_off_epochs.npy'
np.save(filename, result)